### Save and Load the Pruned Models.

1. Install [joblib](https://joblib.readthedocs.io/en/latest/installing.html)
2. To save the model: `joblib.dump(net.cpu(), 'path-to-file')`
3. To load the model: `net = joblib.load('path-to-file').cuda()`

In [1]:
''' setting before run. every notebook should include this code. '''
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="9"

import sys

_r = os.getcwd().split('/')
_p = '/'.join(_r[:_r.index('gate-decorator-pruning')+1])
print('Change dir from %s to %s' % (os.getcwd(), _p))
os.chdir(_p)
sys.path.append(_p)

Change dir from /root/code/gate-decorator-pruning/run/load_pruned_model to /root/code/gate-decorator-pruning


In [2]:
import joblib
net = joblib.load('./ckps/pruned_models/vgg16_cifar10_0.3M.ckp')

net

Parsing config file...
** Assert in demo mode. **


usage: ipykernel_launcher.py [-h] [--config CONFIG]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-cc6e0d91-9cde-4ca8-a895-ae9f9508c4db.json


DataParallel(
  (module): VGG(
    (features): Sequential(
      (0): Conv2d(3, 17, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(17, 46, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(46, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(46, 57, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(57, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(57, 71, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(71, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dil

-----

### Test the loaded model

In [3]:
from config import parse_from_dict

parse_from_dict({
    "base": {
        "task_name": "vgg16_cifar10",
        "cuda": True,
        "seed": 0,
        "checkpoint_path": "",
        "epoch": 0,
        "multi_gpus": True,
        "fp16": False
    },
    "model": {
        "name": "cifar.vgg16",
        "num_class": 10,
        "pretrained": False
    },
    "train": {
        "trainer": "normal",
        "max_epoch": 160,
        "optim": "sgd",
        "steplr": [
            [80, 0.1],
            [120, 0.01],
            [160, 0.001]
        ],
        "weight_decay": 5e-4,
        "momentum": 0.9,
        "nesterov": False
    },
    "data": {
        "type": "cifar10",
        "shuffle": True,
        "batch_size": 128,
        "test_batch_size": 128,
        "num_workers": 4
    },
    "loss": {
        "criterion": "softmax"
    },
    "gbn": {
        "sparse_lambda": 1e-3,
        "flops_eta": 0,
        "lr_min": 1e-3,
        "lr_max": 1e-2,
        "tock_epoch": 10,
        "T": 10,
        "p": 0.005,
        "finetune_epoch": 40
    }
})
from config import cfg
import config

In [4]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim

from logger import logger
from main import set_seeds, recover_pack, adjust_learning_rate, _step_lr, _sgdr
from models import get_model
from utils import dotdict

from prune.universal import Meltable, GatedBatchNorm2d, Conv2dObserver, IterRecoverFramework, FinalLinearObserver
from prune.utils import analyse_model, finetune

In [5]:
set_seeds()
pack = recover_pack()

pack.net = net.cuda()

==> Preparing Cifar10 data..
Files already downloaded and verified
Files already downloaded and verified


AttributeError: 'NoneType' object has no attribute 'model'

In [6]:
cfg

{'base': {'task_name': 'vgg16_cifar10',
  'cuda': True,
  'seed': 0,
  'checkpoint_path': '',
  'epoch': 0,
  'multi_gpus': True,
  'fp16': False},
 'model': {'name': 'cifar.vgg16', 'num_class': 10, 'pretrained': False},
 'train': {'trainer': 'normal',
  'max_epoch': 160,
  'optim': 'sgd',
  'steplr': [[80, 0.1], [120, 0.01], [160, 0.001]],
  'weight_decay': 0.0005,
  'momentum': 0.9,
  'nesterov': False},
 'data': {'type': 'cifar10',
  'shuffle': True,
  'batch_size': 128,
  'test_batch_size': 128,
  'num_workers': 4},
 'loss': {'criterion': 'softmax'},
 'gbn': {'sparse_lambda': 0.001,
  'flops_eta': 0,
  'lr_min': 0.001,
  'lr_max': 0.01,
  'tock_epoch': 10,
  'T': 10,
  'p': 0.005,
  'finetune_epoch': 40}}

In [4]:
pack.trainer.test(pack)

{'test_loss': 0.28081194471709336, 'acc@1': 91.38647151898734}

In [5]:
import torch
import torch.nn as nn
from prune.utils import analyse_model, finetune

In [6]:
flops, params = analyse_model(pack.net.module, torch.randn(1, 3, 32, 32).cuda())

In [10]:
flops / 1024 / 1024

30.01892852783203

In [12]:
params / 1024 / 1024

0.2909698486328125